## 명목임금지수
## https://bigdata-sme.kr/#/datastore/detail/statistics/DATASET_KPC_14

In [2]:
import requests
import pandas as pd
import json
import os
from dotenv import load_dotenv
import time # API 서버 부하를 줄이기 위해 time 라이브러리 추가

# --- dotenv 설정 ---
load_dotenv()
api_key = os.getenv("wage_key")
if not api_key:
    print("❌ 오류: .env 파일에서 'wage_key' 키를 찾을 수 없습니다.")
else:
    url = "https://dataapi.wehago.com/data/"
    headers = {
        "apicode": api_key
    }
    
    # --- 반복문 로직 ---
    all_data_list = []  # 모든 데이터를 저장할 빈 리스트 (장바구니)
    start_index = 0
    chunk_size = 500    # 한 번에 요청할 개수 (0~499는 500개)

    print("데이터 수집을 시작합니다...")

    while True:
        # 1. 'from'과 'to' 파라미터를 동적으로 설정
        params = {
            "dataset_key": "DATASET_KPC_14",
            "from": start_index,
            "to": start_index + chunk_size - 1
        }
        
        print(f"  -> {start_index}부터 {start_index + chunk_size - 1}까지 요청 중...")

        try:
            response = requests.get(url, headers=headers, params=params)

            if response.status_code == 200:
                data = response.json()
                
                # 2. 'data' 키에서 실제 데이터 리스트를 가져옴
                data_chunk = data.get('data') # .get()을 사용하면 'data' 키가 없어도 오류 안남

                # 3. 데이터가 비어있는지 확인
                if not data_chunk:
                    # 서버가 빈 리스트([])를 반환하면, 더 이상 데이터가 없다는 의미
                    print("✅ 모든 데이터를 수집했습니다. 반복을 종료합니다.")
                    break
                
                # 4. (중요) 가져온 데이터를 '장바구니' 리스트에 추가
                all_data_list.extend(data_chunk)
                
                # 5. 다음 루프를 위해 시작 인덱스 업데이트
                start_index += chunk_size
                
                # (권장) API 서버에 대한 예의로, 요청 사이에 짧은 휴식 추가
                time.sleep(0.1) # 0.1초 대기

            else:
                # API 호출 자체가 실패한 경우 (400, 404, 500 등)
                print(f"❌ 오류 발생 (Status Code: {response.status_code})")
                print(response.text)
                break # 오류 발생 시 반복 중단

        except requests.exceptions.RequestException as e:
            print(f"연결 중 오류가 발생했습니다: {e}")
            break # 오류 발생 시 반복 중단

    # --- 반복문 종료 후 ---
    if all_data_list:
        # 6. '장바구니'에 모아둔 모든 데이터를 하나의 DataFrame으로 변환
        df_total = pd.DataFrame(all_data_list)
        
        print("\n--- 최종 DataFrame 변환 결과 ---")
        print(f"총 {len(df_total)}개의 행(row)을 가져왔습니다.")
        print(df_total.head())
    else:
        print("\n가져온 데이터가 없습니다.")



데이터 수집을 시작합니다...
  -> 0부터 499까지 요청 중...
✅ 모든 데이터를 수집했습니다. 반복을 종료합니다.

가져온 데이터가 없습니다.


In [2]:
df_total.to_csv('명목임금지수.csv')

NameError: name 'df_total' is not defined

In [3]:
import requests
import pandas as pd
import json
import os
from dotenv import load_dotenv
import time

# --- dotenv 설정 ---
load_dotenv()
api_key = os.getenv("wage_key")

# 1. API 키 로딩 확인 (가장 중요)
if not api_key:
    print("❌ 오류: .env 파일에서 'wage_key' 키를 찾을 수 없습니다.")
    print(" -> .env 파일 위치와 'wage_key' 이름을 확인하세요.")
else:
    print(f"✅ API 키 로딩 성공 (앞 4자리: {api_key[:4]}****)")
    
    url = "https://dataapi.wehago.com/data/"
    headers = {
        "apicode": api_key
        # (참고) 만약 'apicode'가 아니라면 API 문서를 확인하여 아래와 같이 수정
        # "Authorization": f"Bearer {api_key}" 
        # "X-API-KEY": api_key                
    }
    
    all_data_list = [] 
    start_index = 0
    chunk_size = 500

    print("데이터 수집을 시작합니다...")

    while True:
        params = {
            "dataset_key": "DATASET_KPC_14",
            "from": start_index,
            "to": start_index + chunk_size - 1
        }
        
        print(f" ➡️ {start_index}부터 {start_index + chunk_size - 1}까지 요청 중...")

        try:
            response = requests.get(url, headers=headers, params=params)

            # 2. API 인증 및 호출 확인
            if response.status_code == 200:
                try:
                    data = response.json()
                except requests.exceptions.JSONDecodeError:
                    print("❌ 오류: API가 200 OK를 반환했지만, 응답이 JSON 형식이 아닙니다.")
                    print(f" -> 응답 내용 (첫 200자): {response.text[:200]}")
                    break

                # 3. JSON 'data' 키 확인 (첫 번째 루프에서만 확인)
                if start_index == 0 and 'data' not in data:
                    print(f"⚠️ 경고: JSON 응답에 'data' 키가 없습니다. (첫 요청)")
                    print(f" -> 실제 반환된 키 목록: {list(data.keys())}")
                    print(" -> 'data_chunk = data.get('data')' 부분을 실제 키로 수정해야 할 수 있습니다.")
                
                data_chunk = data.get('data') 

                if not data_chunk:
                    print("✅ 모든 데이터를 수집했거나, 첫 응답의 'data'가 비어있습니다. 반복을 종료합니다.")
                    break
                
                all_data_list.extend(data_chunk)
                start_index += chunk_size
                time.sleep(0.1)

            else:
                # API 호출 자체가 실패한 경우 (401, 403, 404, 500 등)
                print(f"❌ 오류 발생 (Status Code: {response.status_code})")
                try:
                    # 오류 메시지도 JSON일 수 있음
                    error_data = response.json()
                    print(f" -> API 오류 메시지: {error_data}")
                except requests.exceptions.JSONDecodeError:
                    print(f" -> API 오류 응답 (Non-JSON): {response.text}")
                print(" -> (확인) API 키가 정확한지, 헤더('apicode')가 맞는지 확인하세요.")
                break # 오류 발생 시 반복 중단

        except requests.exceptions.RequestException as e:
            print(f"네트워크 연결 중 오류가 발생했습니다: {e}")
            break # 오류 발생 시 반복 중단

    # --- 반복문 종료 후 ---
    if all_data_list:
        df_total = pd.DataFrame(all_data_list)
        print("\n--- ⭐️ 최종 DataFrame 변환 성공 ⭐️ ---")
        print(f"총 {len(df_total)}개의 행(row)을 가져왔습니다.")
        print(df_total.head())
    else:
        print("\n--- ⚠️ 최종 DataFrame 변환 실패 ---")
        print("가져온 데이터가 없습니다. (all_data_list가 비어있음)")
        print(" -> 위의 로그를 확인하여 문제를 해결하세요.")

✅ API 키 로딩 성공 (앞 4자리: 222c****)
데이터 수집을 시작합니다...
 ➡️ 0부터 499까지 요청 중...
✅ 모든 데이터를 수집했거나, 첫 응답의 'data'가 비어있습니다. 반복을 종료합니다.

--- ⚠️ 최종 DataFrame 변환 실패 ---
가져온 데이터가 없습니다. (all_data_list가 비어있음)
 -> 위의 로그를 확인하여 문제를 해결하세요.


In [4]:
import requests
import pandas as pd
import json
import os
from dotenv import load_dotenv
import time

# --- dotenv 설정 ---
load_dotenv()
api_key = os.getenv("wage_key")

# 1. API 키 로딩 확인
if not api_key:
    print("❌ 오류: .env 파일에서 'wage_key' 키를 찾을 수 없습니다.")
    print(" -> .env 파일 위치와 'wage_key' 이름을 확인하세요.")
else:
    print(f"✅ API 키 로딩 성공 (앞 4자리: {api_key[:4]}****)")
    
    url = "https://dataapi.wehago.com/data/"
    headers = {
        "apicode": api_key
    }
    
    # --- 단일 요청 설정 ---
    # while 루프 대신 0부터 5개 (0~4)만 요청해봅니다.
    params = {
        "dataset_key": "DATASET_KPC_14",
        "from": 0,
        "to": 4  # 0, 1, 2, 3, 4 (총 5개)
    }

    print("\n데이터 수집을 1회만 시도합니다...")
    print(f" -> 요청 URL: {url}")
    print(f" -> 요청 Params: {params}")

    try:
        response = requests.get(url, headers=headers, params=params)

        # 2. API 인증 및 호출 확인
        if response.status_code == 200:
            print("\n--- ✅ API 호출 성공 (Status Code: 200) ---")
            try:
                # JSON 응답을 파싱(변환)합니다.
                data = response.json()
                
                # 파싱된 JSON 데이터를 예쁘게 출력합니다.
                print("--- ⭐️ 수신된 JSON 데이터 (전체) ---")
                print(json.dumps(data, indent=2, ensure_ascii=False))
                print("---------------------------------")
                
                # (중요) 'data' 키가 있는지 확인
                if 'data' in data:
                    print("\n[진단] ✅ JSON 응답에 'data' 키가 존재합니다.")
                else:
                    print(f"\n[진단] ⚠️ JSON 응답에 'data' 키가 없습니다!")
                    print(f" -> 실제 키 목록: {list(data.keys())}")
                    
            except requests.exceptions.JSONDecodeError:
                print("❌ 오류: API가 200 OK를 반환했지만, 응답이 JSON 형식이 아닙니다.")
                print(f" -> 응답 내용 (첫 200자): {response.text[:200]}")

        else:
            # API 호출 자체가 실패한 경우 (401, 403, 404, 500 등)
            print(f"\n--- ❌ API 호출 실패 (Status Code: {response.status_code}) ---")
            try:
                error_data = response.json()
                print(f" -> API 오류 메시지: {error_data}")
            except requests.exceptions.JSONDecodeError:
                print(f" -> API 오류 응답 (Non-JSON): {response.text}")
            print(" -> (확인) API 키가 정확한지, 헤더('apicode')가 맞는지 확인하세요.")

    except requests.exceptions.RequestException as e:
        print(f"네트워크 연결 중 오류가 발생했습니다: {e}")

✅ API 키 로딩 성공 (앞 4자리: 222c****)

데이터 수집을 1회만 시도합니다...
 -> 요청 URL: https://dataapi.wehago.com/data/
 -> 요청 Params: {'dataset_key': 'DATASET_KPC_14', 'from': 0, 'to': 4}

--- ✅ API 호출 성공 (Status Code: 200) ---
--- ⭐️ 수신된 JSON 데이터 (전체) ---
{
  "schema": {
    "fields": [
      {
        "number": 1,
        "column_code": "SN",
        "column_name": "순번",
        "data_type": "decimal",
        "column_desc": "자체 시리얼한 번호"
      },
      {
        "number": 2,
        "column_code": "SINGU_CODE_NM",
        "column_name": "신구 코드 명",
        "data_type": "string",
        "column_desc": "표준산업분류코드가 과거버전인지 신규버전인지 확인용"
      },
      {
        "number": 3,
        "column_code": "WAGE_NM",
        "column_name": "임금 명",
        "data_type": "string",
        "column_desc": "명목임금을 코드화 한 값"
      },
      {
        "number": 4,
        "column_code": "INDUST_CL_NM",
        "column_name": "산업 분류 명",
        "data_type": "string",
        "column_desc": "1인당, 시간당 분류를 코드화"
      },
      {
      

In [6]:
import requests
import pandas as pd
import json
import os
from dotenv import load_dotenv
import time

# --- dotenv 설정 ---
load_dotenv()
api_key = os.getenv("wage_key")

# 1. API 키 로딩 확인
if not api_key:
    print("❌ 오류: .env 파일에서 'wage_key' 키를 찾을 수 없습니다.")
else:
    print(f"✅ API 키 로딩 성공 (앞 4자리: {api_key[:4]}****)")
    
    url = "https://dataapi.wehago.com/data/"
    headers = {
        "apicode": api_key
    }
    
    all_data_list = []  # 모든 데이터를 저장할 '장바구니'
    start_index = 0
    chunk_size = 500    # 한 번에 500개씩 요청

    print("데이터 전체 수집을 시작합니다...")

    while True:
        params = {
            "dataset_key": "DATASET_KPC_14",
            "from": start_index,
            "to": start_index + chunk_size - 1
        }
        
        print(f" ➡️ {start_index}부터 {start_index + chunk_size - 1}까지 요청 중...")

        try:
            response = requests.get(url, headers=headers, params=params)

            if response.status_code == 200:
                data = response.json()
                
                # 'data' 키에서 실제 데이터 리스트를 가져옴
                # (테스트에서 'data' 키가 있음을 확인했으므로 .get() 사용)
                data_chunk = data.get('data') 

                # 3. 데이터가 비어있는지 확인
                if not data_chunk:
                    # 서버가 빈 리스트([])를 반환하면, 더 이상 데이터가 없다는 의미
                    print("✅ 모든 데이터를 수집했습니다. 반복을 종료합니다.")
                    break
                
                # 4. (중요) 가져온 데이터를 '장바구니' 리스트에 추가
                all_data_list.extend(data_chunk)
                
                # 5. 다음 루프를 위해 시작 인덱스 업데이트
                start_index += chunk_size
                
                # (권장) API 서버에 대한 예의로, 요청 사이에 짧은 휴식 추가
                time.sleep(0.1) # 0.1초 대기

            else:
                # API 호출 자체가 실패한 경우 (401, 403, 500 등)
                print(f"❌ 오류 발생 (Status Code: {response.status_code})")
                print(response.text)
                break # 오류 발생 시 반복 중단

        except requests.exceptions.RequestException as e:
            print(f"네트워크 연결 중 오류가 발생했습니다: {e}")
            break # 오류 발생 시 반복 중단

    # --- 반복문 종료 후 ---
    if all_data_list:
        # 6. '장바구니'에 모아둔 모든 데이터를 하나의 DataFrame으로 변환
        df_total = pd.DataFrame(all_data_list)
        
        csv_filename = "total_data_output.csv"
        df_total.to_csv(csv_filename, index=False, encoding='utf-8-sig')
        
        print(f"\n--- ⭐️ 최종 DataFrame 변환 및 CSV 저장 성공 ⭐️ ---")
        print(f"총 {len(df_total)}개의 행(row)을 가져왔습니다.")
        print(f"파일이 {csv_filename} 으로 저장되었습니다.")
        print(df_total.head())
    else:
        print("\n--- ⚠️ 최종 수집 실패 ---")
        print("가져온 데이터가 없습니다. (all_data_list가 비어있음)")
        print(" -> API 서버에 해당 dataset_key로 조회할 데이터가 0건일 수 있습니다.")

✅ API 키 로딩 성공 (앞 4자리: 222c****)
데이터 전체 수집을 시작합니다...
 ➡️ 0부터 499까지 요청 중...
✅ 모든 데이터를 수집했습니다. 반복을 종료합니다.

--- ⚠️ 최종 수집 실패 ---
가져온 데이터가 없습니다. (all_data_list가 비어있음)
 -> API 서버에 해당 dataset_key로 조회할 데이터가 0건일 수 있습니다.
